In [ ]:
import tkinter as tk
from tkinter import messagebox as mbox
import pandas as pd
import numpy as np
from tkinter import StringVar
from tkinter import messagebox
import glob
import configparser
import os

#----------------------------------------------------------------
def od_max():
    get_year = entry_year.get() # 年
    get_month = entry_month.get() # 月
    get_date = ra_var.get() # 曜日区分（0,1,2で値を取得)
    color = ""
    if get_date == 0:
        color = "平日.csv"
        col = "_白_最大平均.csv"
    elif get_date == 1:
        color = "平１.csv"
        col = "_黄_最大平均.csv"
    else:
        color = "土曜.csv"
        col = "_赤_最大平均.csv"
    
    if len(listbox.curselection())==0 or get_year=="" or get_month=="":
        messagebox.showinfo("エラー", "未選択項目があります")
        return
    else:
        # Tukaデータのフォルダパス
        tuka = entry_tuka.get()
        # 出力用ファイルパス
        file_put = entry_puts.get()

        
        # 路線
        for get_root in listbox.curselection():
            root = listbox.get(get_root)
            export = file_put + "/" + get_year + get_month + "_" + root + col

            #########################################


            # 赤ダイヤの場合、結合したものをfileとして集計する
            if get_date == 2:
                file_doyou = glob.glob(tuka + "/" + get_year + get_month + "/" + root + "_????????" + "土曜.csv")
                file_kyujitu = glob.glob(tuka + "/" + get_year + get_month + "/" + root + "_????????" + "休日.csv")
                file_kyu1 = glob.glob(tuka + "/" + get_year + get_month + "/" + root + "_????????" + "休１.csv")
                for file_doyou in file_doyou:
                    file_doyou = file_doyou
                for file_kyujitu in file_kyujitu:
                    file_kyujitu = file_kyujitu
                for file_kyu1 in file_kyu1:
                    file_kyu1 = file_kyu1
                df_doyou = pd.read_csv(file_doyou, encoding="shift-jis")
                df_kyujitu = pd.read_csv(file_kyujitu, encoding="shift-jis")
                df_kyu1 = pd.read_csv(file_kyu1, encoding="shift-jis")

                # 休1（日祝）と休日（土曜閉校）をマージして、出力先に出力する
                f=pd.merge(df_kyu1, df_kyujitu, on=[df_kyu1.columns[0],df_kyu1.columns[1],df_kyu1.columns[2],df_kyu1.columns[3],df_kyu1.columns[4],df_kyu1.columns[5],df_kyu1.columns[6],df_kyu1.columns[7],df_kyu1.columns[8],df_kyu1.columns[9],df_kyu1.columns[10],df_kyu1.columns[11],df_kyu1.columns[12],df_kyu1.columns[13]], suffixes=["do", "dhe"], how='outer')
                f.to_csv(export, index=False, encoding='shift-jis')

                # 出力されたものを読み込んで、土曜をマージする　→　出力先に出力
                df_already = pd.read_csv(export, encoding="shift-jis")
                f=pd.merge(df_already, df_doyou, on=[df_already.columns[0],df_already.columns[1],df_already.columns[2],df_already.columns[3],df_already.columns[4],df_already.columns[5],df_already.columns[6],df_already.columns[7],df_already.columns[8],df_already.columns[9],df_already.columns[10],df_already.columns[11],df_already.columns[12],df_already.columns[13]], suffixes=['', 'ni'], how='outer')
                f.to_csv(export, index=False, encoding='shift-jis')

                # 出力されたものを使いたいので、"file"を出力先ファイルにする
                file = export

            else:
                file = glob.glob(tuka + "/" + get_year + get_month + "/" + root + "_????????" + color)
                for file in file:
                    file = file

            #########################################
            
            if os.path.exists(file):

                df = pd.read_csv(file, encoding="shift-jis", header=None)

                # 日付のコピー
                for head_col in range(14, len(df.columns), 6):
                    head_date = df.iat[0,head_col]
                    df.iat[0,head_col+1] = head_date
                    df.iat[0,head_col+2] = head_date
                    df.iat[0,head_col+3] = head_date
                    df.iat[0,head_col+4] = head_date
                    df.iat[0,head_col+5] = head_date

                # 欠損値に0を入れる
                df = df.fillna(0)

                # 通過人員以外の列を削除
                list = ['乗車人数', '降車人数', '通過時刻1', '通過時刻2', '遅れ']
                for head_col in reversed(range(14, len(df.columns))):
                    if df.iat[1, head_col] in list:
                        df.drop(df.columns[head_col], axis='columns', inplace=True)

                # ”通過人員”のヘッダーを日付に変更し、0行目を削除
                for head_col in range(14, len(df.columns)):
                    head_date = df.iat[0, head_col]
                    df.iat[1, head_col] = head_date
                df.drop(df.index[0], axis=0, inplace=True)

                # csv出力 & 再読み込み
                    #本来、再読み込みはせず、以下の２行で書きたい
                    ## df.columns = df.iloc[0]
                    ## df = df.drop(df.index[0])
                df.to_csv(export, encoding='shift-jis', header=1)
                df = pd.read_csv(export, encoding='shift_jis', header=1)
                df.drop(df.columns[0], axis=1, inplace=True) #先頭の数字行を削除
                df.drop(columns=["停留所No", '通過順序', '着予定時刻', '発予定時刻', '往復'], axis=1, inplace=True)

                # グループ化して最大値を取得
                df = df.groupby(['上下','時間帯','基準時刻','営業所No','コースNo','系統No']).max()

                # 最大値の平均を取得
                df.insert(3, '最大通過平均', df.loc[:,df.columns.get_level_values(0).str.contains('月')].mean(axis=1).round().astype(int))

                # 全データをint型にする
                list = []
                for i in df.columns.values:
                    list.append(i)
                del list[0:4]

                for i in list:
                    df[i] = df[i].astype(int)

                # 停留所カラム名変更
                df = df.rename(columns={'停留所名称':'最大停留所名'})

                # csv出力して終了
                df.to_csv(export, encoding='shift_jis', header=1)
            else:
                continue
    
        messagebox.showinfo("最大通過の平均", "終了しました")

    

#----------------------------------------------------------------
    
    
def od_ave():
    get_year = entry_year.get() # 年
    get_month = entry_month.get() # 月
    get_date = ra_var.get() # 曜日区分（0,1,2で値を取得)
    color = ""
    if get_date == 0:
        color = "平日.csv"
        col = "_白_OD.csv"
    elif get_date == 1:
        color = "平１.csv"
        col = "_黄_OD.csv"
    else:
        color = "土曜.csv"
        col = "_赤_OD.csv"
    
    if len(listbox.curselection())==0 or get_year=="" or get_month=="":
        messagebox.showinfo("エラー", "未選択項目があります")
        return
    else:
        # Tukaデータのフォルダパス
        tuka = entry_tuka.get()
        # 出力用ファイルパス
        file_put = entry_puts.get()

        
        # 路線
        for get_root in listbox.curselection():
            root = listbox.get(get_root)
            export = file_put + "/" + get_year + get_month + "_" + root + col

            #########################################


            # 赤ダイヤの場合、結合したものをfileとして集計する
            if get_date == 2:
                file_doyou = glob.glob(tuka + "/" + get_year + get_month + "/" + root + "_????????" + "土曜.csv")
                file_kyujitu = glob.glob(tuka + "/" + get_year + get_month + "/" + root + "_????????" + "休日.csv")
                file_kyu1 = glob.glob(tuka + "/" + get_year + get_month + "/" + root + "_????????" + "休１.csv")

                for file_doyou in file_doyou:
                    file_doyou = file_doyou
                for file_kyujitu in file_kyujitu:
                    file_kyujitu = file_kyujitu
                for file_kyu1 in file_kyu1:
                    file_kyu1 = file_kyu1
                df_doyou = pd.read_csv(file_doyou, encoding="shift-jis")
                df_kyujitu = pd.read_csv(file_kyujitu, encoding="shift-jis")
                df_kyu1 = pd.read_csv(file_kyu1, encoding="shift-jis")

                # 休1（日祝）と休日（土曜閉校）をマージして、出力先に出力する
                f=pd.merge(df_kyu1, df_kyujitu, on=[df_kyu1.columns[0],df_kyu1.columns[1],df_kyu1.columns[2],df_kyu1.columns[3],df_kyu1.columns[4],df_kyu1.columns[5],df_kyu1.columns[6],df_kyu1.columns[7],df_kyu1.columns[8],df_kyu1.columns[9],df_kyu1.columns[10],df_kyu1.columns[11],df_kyu1.columns[12],df_kyu1.columns[13]], suffixes=["do", "dhe"], how='outer')
                f.to_csv(export, index=False, encoding='shift-jis')

                # 出力されたものを読み込んで、土曜をマージする　→　出力先に出力
                df_already = pd.read_csv(export, encoding="shift-jis")
                f=pd.merge(df_already, df_doyou, on=[df_already.columns[0],df_already.columns[1],df_already.columns[2],df_already.columns[3],df_already.columns[4],df_already.columns[5],df_already.columns[6],df_already.columns[7],df_already.columns[8],df_already.columns[9],df_already.columns[10],df_already.columns[11],df_already.columns[12],df_already.columns[13]], suffixes=['', 'ni'], how='outer')
                f.to_csv(export, index=False, encoding='shift-jis')

                # 出力されたものを使いたいので、"file"を出力先ファイルにする
                file = export

            else:
                file = glob.glob(tuka + "/" + get_year + get_month + "/" + root + "_????????" + color)
                for file in file:
                    file = file

            #########################################
            
            if os.path.isfile(file):
                df = pd.read_csv(file, encoding="shift-jis", header=[0,1])

                # 欠損値に0を入れる
                df = df.fillna(0)

                # 平均値を記入する
                df.insert(14, ('','平均_乗車'), df.loc[:,df.columns.get_level_values(1).str.contains('乗車人数')].mean(axis=1))
                df.insert(15, ('','平均_降車'), df.loc[:,df.columns.get_level_values(1).str.contains('降車人数')].mean(axis=1))
                df.insert(16, ('','平均_通過'), df.loc[:,df.columns.get_level_values(1).str.contains('通過人数')].mean(axis=1))
                df.insert(17, ('','平均_遅れ'), df.loc[:,df.columns.get_level_values(1).str.contains('遅れ')].mean(axis=1))

                # ヘッダー行のunnamedは空欄にしておく
                named = "Unnamed"
                for head_col in range(0, len(df.columns)):
                    if named in df.columns.get_level_values(0)[head_col]:
                        df = df.rename(columns={df.columns.get_level_values(0)[head_col]: ''})

                # 通過時刻1, 通過時刻2の列を削除
                list = ['通過時刻1', '通過時刻2', '着予定時刻', '往復']
                for i in list:
                    df.drop(('',i), axis=1, inplace=True)

                # csv出力して終了
                df.to_csv(export, encoding='shift-jis', header=2, index=None)
            else:
                continue
    
    messagebox.showinfo("便ごとの詳細", "終了しました")
    
def save():
    global conf
    # data セクションが無かったら追加
    if not conf.has_section('data'):
        conf.add_section('data')
    # EntryBoxの値を設定ファイルに書き込む
    conf.set('data', 'tuka', entry_tuka.get())
    conf.set('data', 'puts', entry_puts.get())
    with open(CONFIG_FILE,'w') as f:
        conf.write(f)
    messagebox.showinfo("マスタの保存", "保存が完了しました")

#----------------------------------------------------------------  

# configファイルの読み込み
CONFIG_FILE = 'config.ini' #configファイルパスは任意で変更
conf = configparser.ConfigParser()
conf.read(CONFIG_FILE)

# ウィンドウを作成 --- (*1)
win = tk.Tk()
win.geometry("500x500") # サイズを指定
win.title("OD分析ツール")

# テキストボックス（年）
entry_year = tk.Entry(win, width=5)
entry_year.place(x=25, y=10)
label_year = tk.Label(win, text='年')
label_year.place(x=65, y=13)

# テキストボックス（月）
entry_month = tk.Entry(width=5)
entry_month.place(x=90, y=10)
label_month = tk.Label(win, text='月')
label_month.place(x=130, y=13)

# リストボックス項目（路線の選択）
var = StringVar(value=['11舘山寺', '13志都呂宇布見', '14大久保', '15伊佐見', '17遠州浜蜆塚', '19大平台', '20中田島泉高丘', '21大塚ひとみヶ丘', '22早出さぎの宮', '23鶴見富塚じゅんかん', '25笠井高台', '26磐田山の手', '27医大じゅんかん', '28都田', '29気賀三ヶ日', '30引佐', '31渋川', '32奥山', '34蒲小沢渡', '36内野台', '37浜北医大三方原聖隷', '41秋葉', '42鹿島', '44磐田天竜', '45上島イオン市野', '50掛塚さなる台', '51城之崎', '53浜名', '56磐田市立病院福田', '59萩丘都田', '61引佐町営（奥山循環）', '64浅田米津', '67和合西山', '68三島江之島', '72北遠本線'])
# リストボックスの生成
listbox = tk.Listbox(listvariable=var, width=25, height=15)
listbox.configure(selectmode='extended')
listbox.place(x=25, y=80)

# ラジオボタンの配置
ra_var = tk.IntVar()
rdo1 = tk.Radiobutton(win, value=0, text='平日開校', variable=ra_var)
rdo1.place(x=200, y=80)

rdo2 = tk.Radiobutton(win, value=1, text='平日閉校', variable=ra_var)
rdo2.place(x=200, y=110)

rdo3 = tk.Radiobutton(win, value=2, text='土日祝', variable=ra_var)
rdo3.place(x=200, y=140)
    
okButton = tk.Button(win, text='最大通過の平均', command=od_max)
okButton.place(x=300, y=100)

okButton = tk.Button(win, text='便ごとの詳細　', command=od_ave)
okButton.place(x=300, y=150)

# マスタのラベル
label_master = tk.Label(win, text='----------マスタ（ファイルパスの「\」は「/」に直すこと）----------')
label_master.place(x=20, y=360)

# テキストボックス（Tukaデータのパス）
entry_tuka = tk.Entry(win, width=50)
entry_tuka.insert(0, conf.get('data', 'tuka', fallback='初期値'))
entry_tuka.place(x=120, y=390)
label_tuka = tk.Label(win, text='Tukaファイルパス')
label_tuka.place(x=20, y=390)


# テキストボックス（出力先のパス）
entry_puts = tk.Entry(win, width=50)
entry_puts.insert(0, conf.get('data', 'puts', fallback='初期値'))
entry_puts.place(x=120, y=410)
label_puts = tk.Label(win, text='出力先')
label_puts.place(x=20, y=410)

# マスタボタンの作成
button_master = tk.Button(win, text='マスタの保存', command=save)
button_master.place(x=350, y=440)

#----------------------------------------------------------------



# ウィンドウを動かす
win.mainloop()